In [1]:
import pandas as pd
import numpy as np
from GPyOpt.methods.bayesian_optimization import BayesianOptimization
from model import VAE

In [2]:
pairs_dev_train_path = '../pairsDevTrain.txt'
pairs_dev_test_path = '../pairsDevTest.txt'

def extract_pairs_for_development(pairs_dev_filepath: str):
    pairs = []
    with open(pairs_dev_filepath, 'r') as f:
        size = int(next(f))
        for i in range(size):
            line = next(f)
            name, num1, num2 = line.strip().split()
            pairs.append(((name, int(num1)), (name, int(num2))))
        for i in range(size):
            line = next(f)
            name1, num1, name2, num2 = line.strip().split()
            pairs.append(((name1, int(num1)), (name2, int(num2))))
    return pairs

pairs_dev_train = extract_pairs_for_development(pairs_dev_train_path)
pairs_dev_test = extract_pairs_for_development(pairs_dev_test_path)

In [3]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

dataset_path = '../lfw/'

class PairDataGenerator(Dataset):
    def __init__(self, pairs, transform=None):
        self.pairs = pairs

        composition = []
        if transform:
            composition.append(transform)
        composition.append(transforms.ToTensor())

        self.transform = transforms.Compose(composition)

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, index):
        (name1, num1), (name2, num2) = self.pairs[index]
        # Assuming images are in a folder named 'lfw-deepfunneled'
        image_path1 = f'{dataset_path}/{name1}/{name1}_{num1:0>4}.jpg'
        image_path2 = f'{dataset_path}/{name2}/{name2}_{num2:0>4}.jpg'

        image1 = Image.open(image_path1).convert("RGB")
        image2 = Image.open(image_path2).convert("RGB")

        image1 = self.transform(image1)
        image2 = self.transform(image2)

        y_true = 1 if name1 == name2 else 0
        return image1, image2, y_true

In [4]:
train_pair_gen = PairDataGenerator(pairs_dev_train, transform=transforms.Resize((128,128)))
test_pair_gen = PairDataGenerator(pairs_dev_test, transform=transforms.Resize((128,128)))
train_pair_loader = DataLoader(train_pair_gen, batch_size=32)
test_pair_loader = DataLoader(test_pair_gen, batch_size=32)

In [5]:
vae = VAE((3, 128, 128), 1024, 6).cuda()
vae.load_state_dict(torch.load('trained/vae-(128, 128)-1024-6-1.dat')['state_dict'])
vae.eval()

VAE(
  (encoder_conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
    (9): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.01)
    (12): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (13): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [6]:
def to_numpy(pair_loader):
    left_latent_list = []
    right_latent_list = []
    value_list = []
    for left_images, right_images, value_batch in pair_loader:
        left_images = left_images.cuda()
        right_images = right_images.cuda()
        left_images_latent = vae.forward_encoder(left_images)[0].detach().cpu().numpy()
        right_images_latent = vae.forward_encoder(right_images)[0].detach().cpu().numpy()
        values = value_batch.detach().cpu().numpy()
        left_latent_list.append(left_images_latent)
        right_latent_list.append(right_images_latent)
        value_list.append(values)
    left_latent = np.concatenate(left_latent_list, axis=0)
    right_latent = np.concatenate(right_latent_list, axis=0)
    values = np.concatenate(value_list, axis=0)
    return left_latent, right_latent, values
train_left_latent, train_right_latent, train_values = to_numpy(train_pair_loader)
test_left_latent, test_right_latent, test_values = to_numpy(test_pair_loader)

In [7]:
dif = train_left_latent - train_right_latent
sq_dif = dif ** 2
sq_dif_sum = np.sum(sq_dif, axis=1)
train_dist_array = np.sqrt(sq_dif_sum)
train_min = train_dist_array.min()
train_max = train_dist_array.max()
print(train_min)
print(train_max)
print(len(train_dist_array))
train_dist_array

0.37195048
4.7118587
2200


array([1.4190643, 2.0344756, 3.2802236, ..., 2.3242188, 3.1788743,
       1.5478997], dtype=float32)

In [8]:
dif = test_left_latent - test_right_latent
sq_dif = dif ** 2
sq_dif_sum = np.sum(sq_dif, axis=1)
test_dist_array = np.sqrt(sq_dif_sum)
test_min = test_dist_array.min()
test_max = test_dist_array.max()
print(test_min)
print(test_max)
print(len(test_dist_array))
test_dist_array

0.52522093
4.7207284
1000


array([2.9992962 , 2.3960783 , 3.4272218 , 2.717269  , 2.5799737 ,
       3.6390877 , 1.645507  , 2.3753755 , 2.519557  , 2.1207674 ,
       2.4927742 , 1.5020404 , 2.215817  , 1.747654  , 2.193992  ,
       3.008318  , 1.7492172 , 3.1234841 , 1.2696934 , 1.4292512 ,
       2.0696828 , 2.4238434 , 1.3058199 , 1.9354028 , 1.9576133 ,
       3.2896333 , 2.1647182 , 1.9686898 , 0.86193967, 1.8225218 ,
       1.1332887 , 1.6272203 , 1.5958564 , 2.4536881 , 1.1353883 ,
       3.0961926 , 3.3706794 , 2.233334  , 1.9191918 , 2.8544586 ,
       1.0093877 , 2.6155488 , 3.159827  , 1.552377  , 2.805454  ,
       2.764229  , 2.4262052 , 1.7498633 , 2.7891903 , 2.4301276 ,
       1.717214  , 2.3717334 , 2.968008  , 3.4440844 , 2.8863044 ,
       1.9704925 , 3.0784292 , 1.6713713 , 2.3905447 , 2.3002608 ,
       2.8493576 , 3.2083232 , 2.920108  , 1.7480929 , 1.6659873 ,
       3.3518717 , 1.957069  , 1.5191396 , 1.6495969 , 1.6365972 ,
       1.860617  , 1.0299094 , 1.8124154 , 2.2888606 , 2.28021

In [9]:
def obj_func(threshold: float) -> float:
    decision = (train_dist_array < threshold).astype(int)
    accuracy = np.mean(decision == train_values)
    return -accuracy
def evaluate(dist_array: np.array, true: np.array, threshold: float) -> float:
    decision = (dist_array < threshold).astype(int)
    return np.mean(decision == true)

In [10]:
domain = [{'name': 'threshold', 'type': 'continuous', 'domain': (train_min, train_max)}]
max_iter = 50
BO = BayesianOptimization(f = obj_func, domain = domain)
BO.run_optimization(max_iter=max_iter)

In [11]:
opt_values = BO.x_opt
print(opt_values)

[2.13008393]


In [12]:
obj_func(*opt_values)

-0.5681818181818182

In [13]:
evaluate(test_dist_array, test_values, threshold=opt_values[0])

0.559